In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from geopy import distance

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Import json packages to explode json columns like related_same_day_brand
import json
from pandas.io.json import json_normalize

import requests
from bs4 import BeautifulSoup
import pandas as pd

print('Libraries imported.')

Libraries imported.


In [ ]:
#download the data
# !wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [2]:
# requesting uri
wikilink = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami"
res = requests.get(wikilink)

In [4]:
soup  = BeautifulSoup(res.content, "lxml") # read/parse html-content
wikitable = soup.find_all("table")[0] # get first html-table
miami_df = pd.read_html(str(wikitable))[0] # read/convert to pandas dataframe

In [5]:
# print(miami_df.shape)
# miami_df.dropna(subset=['Coordinates'], inplace=True)
# print(miami_df.shape)
miami_df = miami_df[miami_df['Coordinates'].isna() == False]
miami_df = miami_df[['Neighborhood','Coordinates']]
o = miami_df['Coordinates'].str.split("-", expand=True)
miami_df['Latitude'] = (o[0]).astype(float)
miami_df['Longitude'] = (o[1]).astype(float)
miami_df['Longitude'] = miami_df['Longitude'].apply(lambda i: i*-1)
miami_df.drop(columns=['Coordinates'], inplace=True)
miami_df.insert(0, 'Borough', 'Miami')

In [6]:
miami_df

,Borough,Neighborhood,Latitude,Longitude
0,Miami,Allapattah,25.815,-80.224
1,Miami,Arts & Entertainment District,25.799,-80.190
2,Miami,Brickell,25.758,-80.193
3,Miami,Buena Vista,25.813,-80.192
4,Miami,Coconut Grove,25.712,-80.257
5,Miami,Coral Way,25.750,-80.283
6,Miami,Design District,25.813,-80.193
7,Miami,Downtown,25.774,-80.193
8,Miami,Edgewater,25.802,-80.190
9,Miami,Flagami,25.762,-80.316


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [42]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [43]:
%%time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

CPU times: user 2.77 s, sys: 16 ms, total: 2.78 s
Wall time: 2.77 s


In [44]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan']
neighborhoods = neighborhoods.append(miami_df, sort=True).reset_index(drop=True)
# neighborhoods.drop(columns=['index'], axis=1, inplace=True)

neighborhoods.head(100)

,Borough,Latitude,Longitude,Neighborhood
0,Manhattan,40.876551,-73.910660,Marble Hill
1,Manhattan,40.715618,-73.994279,Chinatown
2,Manhattan,40.851903,-73.936900,Washington Heights
3,Manhattan,40.867684,-73.921210,Inwood
4,Manhattan,40.823604,-73.949688,Hamilton Heights
5,Manhattan,40.816934,-73.957385,Manhattanville
6,Manhattan,40.815976,-73.943211,Central Harlem
7,Manhattan,40.792249,-73.944182,East Harlem
8,Manhattan,40.775639,-73.960508,Upper East Side
9,Manhattan,40.775930,-73.947118,Yorkville


In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 2 boroughs and 64 neighborhoods.


In [46]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [47]:
%%time
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#853ab0',
        fill=True,
        fill_color='#853ab0',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

CPU times: user 40 ms, sys: 4 ms, total: 44 ms
Wall time: 51.2 ms


In [50]:
%%time
map_miami = folium.Map(location=[25.77427, -80.19366], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(miami_df['Latitude'], miami_df['Longitude'], miami_df['Borough'], miami_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#187306',
        fill=True,
        fill_color='#187306',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  
    
map_miami

CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 24.8 ms


In [51]:
# CLIENT_ID = 'U54SVMROBPRP3B0NW1FF5QPCCEPMKZS0PHHUDNUTQ3KBNFX2' # your Foursquare ID
# CLIENT_SECRET = 'Y5302JRTVUIGCWR1EROFFHUIGFFFAD50N2AZ4J4ND3JN1TSK' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version

# AR-data-science auths
CLIENT_ID = 'RU3Y3XCL1D3X4IFWHEPI3VYYAEEGSWMVQTWP2PHZ1DEL1E2R' # your Foursquare ID
CLIENT_SECRET = 'GMMPCVSMWDJDXSOTEO22G3FN4H2BUFSZO05SYEGRGW4N5AKL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# type your answer here
LIMIT = 100
radius = 500
section = 'trending'

Your credentails:
CLIENT_ID: RU3Y3XCL1D3X4IFWHEPI3VYYAEEGSWMVQTWP2PHZ1DEL1E2R
CLIENT_SECRET:GMMPCVSMWDJDXSOTEO22G3FN4H2BUFSZO05SYEGRGW4N5AKL


In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
        
        # make the GET request
        try:
            response = requests.get(url).json()
            results = response['response']['groups'][0]['items']
        except Exception as ex:
            print("{}, problem here {}".format(name, response))
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
%%time
nyc_miami_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood
CPU times: user 2.79 s, sys: 88 ms, total: 2.88 s
Wall time: 34.8 s


In [108]:
# nyc_miami_venues = pd.read_csv('nyc_miami_venues.csv')
print(nyc_miami_venues.shape)
# Foursquare may respond: Quota exceeded, then I have the content....
# other minor problem: Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.
nyc_miami_venues.to_csv(path_or_buf='nyc_miami_venues.csv')
nyc_miami_venues.head(5)
# last response (10343, 7)

# sao paulo -23.5475 -46.63611

(3842, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Five Below,40.877451,-73.905170,Miscellaneous Shop
2,Marble Hill,40.876551,-73.91066,Boston Market,40.877430,-73.905412,American Restaurant
3,Marble Hill,40.876551,-73.91066,SUBWAY,40.878493,-73.905385,Sandwich Place
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [107]:
# AB_NYC_2019.csv
airbnb_df = pd.read_csv('AB_NYC_2019.csv')
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [168]:
%%time

from functools import partial
import multiprocessing as mp
from pandarallel import pandarallel
pandarallel.initialize()
#parallel_apply

# nyc_miami_venues.drop('airbn_near', axis=1, inplace=True)

def near(lat, long):
    count = 0
    print("looking for {}, {}".format(lat,long))
    for idx, ir in airbnb_df.iterrows():
        ila,ilo = ir['latitude'], ir['longitude']
        km = distance.distance((lat,long), (ila,ilo)).km
        if km*1000 < 200:
            count = count + 1
            print("near found **{} [{},{}] [{},{}]".format(km, lat,lng, ila,ilo))         
    return count

nyc_miami_venues['airbnb_near'] = nyc_miami_venues.head(8).parallel_apply(lambda row: near(row['Venue Latitude'], row['Venue Longitude']), axis=1)


# pool = mp.Pool(mp.cpu_count())

# def doit(row):
#     lat, lng = row['Venue Latitude'], row['Venue Longitude']
#     print("looking for {}, {}".format(lat,lng))
#     for idx, ir in airbnb_df.iterrows():
#         ila,ilo = ir['latitude'], ir['longitude']
#         km = distance.distance((lat,lng), (ila,ilo)).km
#         if km*1000 < 200:
#             print("near found **{} [{},{}] [{},{}]".format(km, lat,lng, ila,ilo))

# ok = nyc_miami_venues.head(1)
# pool.map(doit, ok.T.to_dict().values())
# pool.terminate()
# pool.join()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 8 workers
CPU times: user 536 ms, sys: 268 ms, total: 804 ms
Wall time: 1min 19s


In [169]:
nyc_miami_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,airbnb_near
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place,0.000000
1,Marble Hill,40.876551,-73.91066,Five Below,40.877451,-73.905170,Miscellaneous Shop,0.000000
2,Marble Hill,40.876551,-73.91066,Boston Market,40.877430,-73.905412,American Restaurant,0.000000
3,Marble Hill,40.876551,-73.91066,SUBWAY,40.878493,-73.905385,Sandwich Place,0.000000
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop,40.877136
5,Marble Hill,40.876551,-73.91066,Lot Less Closeouts,40.878270,-73.905265,Discount Store,1.000000
6,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner,1.000000
7,Marble Hill,40.876551,-73.91066,The Children's Place,40.873672,-73.908156,Kids Store,1.000000
8,Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop,NaN
9,Marble Hill,40.876551,-73.91066,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium,NaN


In [103]:
nyc_miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,5,5,5,5,5,5
Arts & Entertainment District,17,17,17,17,17,17
Battery Park City,94,94,94,94,94,94
Brickell,54,54,54,54,54,54
Buena Vista,40,40,40,40,40,40
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100


In [58]:
print('There are {} uniques categories.'.format(len(nyc_miami_venues['Venue Category'].unique())))

There are 335 uniques categories.


In [64]:
%%time

# Analyze Each Neighborhood --------------
# one hot encoding
nyc_miami_onehot = pd.get_dummies(nyc_miami_venues[['Venue Category']], prefix="", prefix_sep="")

nyc_miami_onehot.drop('Neighborhood', axis=1, inplace=True)
# with method DataFrame.insert may resume some task.
nyc_miami_onehot.insert(0, 'Neighborhood', nyc_miami_venues['Neighborhood'])
nyc_miami_onehot.head()

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 40.5 ms


,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,S

In [65]:
nyc_miami_onehot.shape

(3842, 335)

In [96]:
%%time

nyc_miami_grouped = nyc_miami_onehot.groupby('Neighborhood').mean().reset_index()
print(nyc_miami_grouped.shape)
nyc_miami_grouped.head()

(63, 335)
CPU times: user 48 ms, sys: 4 ms, total: 52 ms
Wall time: 48.6 ms


,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,S

In [ ]:
%%time
num_top_venues = 5

for hood in nyc_miami_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyc_miami_grouped[nyc_miami_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_miami_grouped['Neighborhood']

for ind in np.arange(nyc_miami_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_miami_grouped.iloc[ind, :], num_top_venues)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head(10)

(63, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allapattah,Convenience Store,Department Store,American Restaurant,Print Shop,Lounge,Yoga Studio,Filipino Restaurant,Fish Market,Flea Market,Flower Shop
1,Arts & Entertainment District,Gym,Restaurant,Cuban Restaurant,Pizza Place,Paper / Office Supplies Store,Spa,Park,Sandwich Place,Salon / Barbershop,Ice Cream Shop
2,Battery Park City,Park,Coffee Shop,Gym,Memorial Site,Italian Restaurant,Wine Shop,Ice Cream Shop,Plaza,Playground,Pizza Place
3,Brickell,Italian Restaurant,Japanese Restaurant,Café,Juice Bar,Sandwich Place,Restaurant,Bank,Bar,Salon / Barbershop,Argentinian Restaurant
4,Buena Vista,Café,Italian Restaurant,Art Museum,Jewelry Store,Nightclub,Boutique,Coffee Shop,Mediterranean Restaurant,Cosmetics Shop,Big Box Store
5,Carnegie Hill,Coffee Shop,Pizza Place,Bar,Café,Cosmetics Shop,Bookstore,Spa,Bakery,Yoga Studio,French Restaurant
6,Central Harlem,African Restaurant,French Restaurant,Chinese Restaurant,American Restaurant,Public Art,Bar,Gym / Fitness Center,Seafood Restaurant,Boutique,Tapas Restaurant
7,Chelsea,Art Gallery,Bakery,Italian Restaurant,Café,Nightclub,Coffee Shop,Ice Cream Shop,Cupcake Shop,Gym / Fitness Center,Cosmetics Shop
8,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Dessert Shop,Optical Shop,Boutique,Italian Restaurant,Malay Restaurant,Massage Studio,Bubble Tea Shop
9,Civic Center,Spa,Coffee Shop,Gym / Fitness Center,Sandwich Place,French Restaurant,American Restaurant,Yoga Studio,Café,Italian Restaurant,Korean Restaurant


<h1>Cluster Neighborhoods</h1>

In [75]:
# set number of clusters
kclusters = 6

nyc_miami_grouped_clustering = nyc_miami_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_miami_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3,
       1, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 3, 3, 3, 3, 3],
      dtype=int32)

In [76]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
nyc_miami_merged = neighborhoods
print("nyc_miami_merged/neighborhoods  {}".format(neighborhoods.shape))
print("neighborhoods_venues_sorted {}".format(neighborhoods_venues_sorted.shape))
    
# join convert some infinite kmeans.labels_ bad bad ....
nyc_miami_merged = nyc_miami_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print("before isfinite {}".format(nyc_miami_merged.shape))
# # nyc_miami_merged['Cluster Labels'].dropna(inplace=True)

for n,h in enumerate(np.isfinite(nyc_miami_merged['Cluster Labels'])):
    if not h:
        print(nyc_miami_merged.values[n])

# clear infinite labels after join         
nyc_miami_merged = nyc_miami_merged[np.isfinite(nyc_miami_merged['Cluster Labels'])]
print("after... isfinite {}".format(nyc_miami_merged.shape))

nyc_miami_merged['Cluster Labels'].dropna(inplace=True)
print("drop NaN {}".format(nyc_miami_merged.shape))
nyc_miami_merged['Cluster Labels'] = nyc_miami_merged['Cluster Labels'].astype(int)

nyc_miami_merged.head(100) # check the last columns!

nyc_miami_merged/neighborhoods  (64, 4)
neighborhoods_venues_sorted (63, 12)
before isfinite (64, 15)
after... isfinite (64, 15)
drop NaN (64, 15)


,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,40.876551,-73.910660,Marble Hill,3,Sandwich Place,Coffee Shop,Yoga Studio,Bank,Discount Store,Department Store,Deli / Bodega,Donut Shop,Pizza Place,Seafood Restaurant
1,Manhattan,40.715618,-73.994279,Chinatown,3,Chinese Restaurant,Cocktail Bar,Bakery,Dessert Shop,Optical Shop,Boutique,Italian Restaurant,Malay Restaurant,Massage Studio,Bubble Tea Shop
2,Manhattan,40.851903,-73.936900,Washington Heights,3,Café,Bakery,Deli / Bodega,Grocery Store,Mobile Phone Shop,Coffee Shop,New American Restaurant,Tapas Restaurant,Mexican Restaurant,Sandwich Place
3,Manhattan,40.867684,-73.921210,Inwood,3,Café,Mexican Restaurant,Lounge,Pizza Place,American Restaurant,Bakery,Spanish Restaurant,Wine Bar,Frozen Yogurt Shop,Chinese Restaurant
4,Manhattan,40.823604,-73.949688,Hamilton Heights,3,Café,Pizza Place,Mexican Restaurant,Coffee Shop,Yoga Studio,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Sushi Restaurant,Park
5,Manhattan,40.816934,-73.957385,Manhattanville,3,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Ramen Restaurant,Sushi Restaurant,Supermarket,Café,Spanish Restaurant
6,Manhattan,40.815976,-73.943211,Central Harlem,3,African Restaurant,French Restaurant,Chinese Restaurant,American Restaurant,Public Art,Bar,Gym / Fitness Center,Seafood Restaurant,Boutique,Tapas Restaurant
7,Manhattan,40.792249,-73.944182,East Harlem,3,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Latin American Restaurant,Dance Studio,Coffee Shop,Seafood Restaurant,Spanish Restaurant,Beer Bar
8,Manhattan,40.775639,-73.960508,Upper East Side,3,Italian Restaurant,Exhibit,Coffee Shop,Art Gallery,Juice Bar,American Restaurant,Bakery,French Restaurant,Mexican Restaurant,Hotel
9,Manhattan,40.775930,-73.947118,Yorkville,3,Italian Restaurant,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Bar,Deli / Bodega,Japanese Restaurant,Wine Shop,Sandwich Place


In [104]:
nyc_zone = nyc_miami_merged[nyc_miami_merged['Borough'] == 'Manhattan']

def draw_map(zone, lat, long):
    # create map
    map_clusters = folium.Map(location=[lat, long], zoom_start=11)
    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []

    for lat, lon, poi, cluster in zip(zone['Latitude'], zone['Longitude'], zone['Neighborhood'], zone['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)    
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    return map_clusters

draw_map(nyc_zone, latitude,longitude)

In [106]:
miami_zone = nyc_miami_merged[nyc_miami_merged['Borough'] == 'Miami']
draw_map(miami_zone, 25.77427, -80.19366)

In [91]:
nyc_miami_merged.loc[nyc_miami_merged['Cluster Labels'] == 3, nyc_miami_merged.columns[[0] + list(range(3, nyc_miami_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,3,Sandwich Place,Coffee Shop,Yoga Studio,Bank,Discount Store,Department Store,Deli / Bodega,Donut Shop,Pizza Place,Seafood Restaurant
1,Manhattan,Chinatown,3,Chinese Restaurant,Cocktail Bar,Bakery,Dessert Shop,Optical Shop,Boutique,Italian Restaurant,Malay Restaurant,Massage Studio,Bubble Tea Shop
2,Manhattan,Washington Heights,3,Café,Bakery,Deli / Bodega,Grocery Store,Mobile Phone Shop,Coffee Shop,New American Restaurant,Tapas Restaurant,Mexican Restaurant,Sandwich Place
3,Manhattan,Inwood,3,Café,Mexican Restaurant,Lounge,Pizza Place,American Restaurant,Bakery,Spanish Restaurant,Wine Bar,Frozen Yogurt Shop,Chinese Restaurant
4,Manhattan,Hamilton Heights,3,Café,Pizza Place,Mexican Restaurant,Coffee Shop,Yoga Studio,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Sushi Restaurant,Park
5,Manhattan,Manhattanville,3,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Ramen Restaurant,Sushi Restaurant,Supermarket,Café,Spanish Restaurant
6,Manhattan,Central Harlem,3,African Restaurant,French Restaurant,Chinese Restaurant,American Restaurant,Public Art,Bar,Gym / Fitness Center,Seafood Restaurant,Boutique,Tapas Restaurant
7,Manhattan,East Harlem,3,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Latin American Restaurant,Dance Studio,Coffee Shop,Seafood Restaurant,Spanish Restaurant,Beer Bar
8,Manhattan,Upper East Side,3,Italian Restaurant,Exhibit,Coffee Shop,Art Gallery,Juice Bar,American Restaurant,Bakery,French Restaurant,Mexican Restaurant,Hotel
9,Manhattan,Yorkville,3,Italian Restaurant,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Bar,Deli / Bodega,Japanese Restaurant,Wine Shop,Sandwich Place


In [ ]:
1